# 3.6 Lab - Linear Regression

## 3.6.1 Importing Packages

We start by importing the needed standard libraries at the top level.

In [2]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from statsmodels.stats.outliers_influence \
    import variance_inflation_factor as VIF
from statsmodels.stats.anova import anova_lm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)

#### Inspecting Objects and Namespaces

The function `dir()` provides a list of objects in a namespace.

In [3]:
dir()

['In',
 'MS',
 'Out',
 'VIF',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'anova_lm',
 'exit',
 'get_ipython',
 'load_data',
 'np',
 'open',
 'pd',
 'poly',
 'quit',
 'sm',
 'subplots',
 'summarize']

This shows everything that `Python` can find at the top level. There are certain objects like `__builtins__` that contain references to built-in functions like `print()`.

Every Python object has its own notion of namespace, also accessible with `dir()`. This will include the attributes of the object as well as any methods associated with it. For example, we can see `sum` in the listing for an array.

In [4]:
A = np.array([3, 5, 11])
dir(A)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_namespace__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__buffer__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',


This indicates that the object `A.sum` exists. IT is a method that can be used to compute the sum of the array `A` as seen typing `A.sum?`.

In [5]:
A.sum?

Docstring:
a.sum(axis=None, dtype=None, out=None, keepdims=False, initial=0, where=True)

Return the sum of the array elements over the given axis.

Refer to `numpy.sum` for full documentation.

See Also
--------
numpy.sum : equivalent function
Type:      builtin_function_or_method

In [6]:
A.sum()

np.int64(19)

## 3.6.2 Simple Linear Regression

In this section we will construct **model matrices** (or **design matrices**) using the `ModelSpec()` transform from `ISLP.models`. We will use the `Boston` dataset (see 2.4 Exercises for full details on the Boston dataset).

The Python model `statsmodel` contains functions implementing several commonly used regression methods. The `load_data()` function in the `ISLP` package loads in the data for us automatically.

In [10]:
Boston = load_data("Boston")
Boston.columns

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'lstat', 'medv'],
      dtype='object')

We start by using the `sm.OLS()` function to fit a simple linear regression model. The response will be `medv` and `lstat` will be the single predictor. For this model, we can create the model matrix by hand.

In [13]:
# Establish the intercept value and extract the lstat predictor
X = pd.DataFrame({'intercept' : np.ones(Boston.shape[0]),
                  'lstat': Boston['lstat']})
X[:4]

,intercept,lstat
0,1.0,4.98
1,1.0,9.14
2,1.0,4.03
3,1.0,2.94


Next, we extract the response and fit the model.

In [14]:
y = Boston['medv']      # Extract the response
model = sm.OLS(y, X)    # Initiate the model
results = model.fit()   # Fit the model

The `ISLP` function `summarize()` produces a simple table of the parameter estimates, their standard errors, t-statistics, and p-values. It takes in a single argument, namely the `results` returned from the `fit` method above.

In [15]:
# Summarize the results
summarize(results)

,coef,std err,t,P>|t|
intercept,34.5538,0.563,61.415,0.0
lstat,-0.9500,0.039,-24.528,0.0


Next we outline a more useful and general framework for constructing a model matrix `X`.

#### Using Transformations: Fit and Transform

The `sklearn` Python package has **transforms**, which are objects that is created to handle datasets to be used in a predictive model. It consists of an object that is created via some parameters passed in as arguments. There are two main methods to this object: `fit()` and `transform()`. 

The transform `ModelSpec()` in the `ISLP` package does some transforms for us in the background, and may perform some data preprocessing before fitting the model. We'll first do an example regressing onto `lstat`. 

In [18]:
design = MS(['lstat'])
design = design.fit(Boston)
X = design.transform(Boston)
X[:4]

,intercept,lstat
0,1.0,4.98
1,1.0,9.14
2,1.0,4.03
3,1.0,2.94


Note in this simple case for simple linear regression, `fit()` merely confirms that `lstat` is within the `Boston` dataset. Then `trasnform()` constructs the model matrix with two columns: an `intercept` and the variable `lstat`.

We can combine these steps with the `fit_transform()` method.

In [22]:
design = MS(['lstat'])
X = design.fit_transform(Boston)
X[:4]

,intercept,lstat
0,1.0,4.98
1,1.0,9.14
2,1.0,4.03
3,1.0,2.94


Returning to our fitted regression model, the object `results` has several methods that can be used for inference. For a full and somewhat exhaustive summary of the fit, we can use the `summary()` method.

In [23]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     601.6
Date:                Thu, 20 Mar 2025   Prob (F-statistic):           5.08e-88
Time:                        20:36:10   Log-Likelihood:                -1641.5
No. Observations:                 506   AIC:                             3287.
Df Residuals:                     504   BIC:                             3295.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     34.5538      0.563     61.415      0.000      33.448      35.659
lstat         -0.9500      0.039    -24.528      0.000      -1.026      -0.874
==============================================================================
Omnibus:                      137.043   Durbin-Watson:                   0.892
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              291.373
Skew:                           1.453   Prob(JB):                     5.36e-64
Kurtosis:                       5.319   Cond. No.                         29.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The fitted coefficients can be retrieved using the `params` attribute of `results`.

In [24]:
results.params

intercept    34.553841
lstat        -0.950049
dtype: float64

The `get_predictions()` method can be used to obtain predictions and produce confidence intervals and prediction intervals for the prediction of `medv` for given values of `lstat`.

First we create a new dataframe containing only the variable `lstat`, and the values for this variable at which we want to make predictions. Then we use `transform()` method of `design` to create the corresponding model matrix.

In [25]:
new_df = pd.DataFrame({'lstat' : [5, 10, 15]})
newX = design.transform(new_df)
newX

,intercept,lstat
0,1.0,5
1,1.0,10
2,1.0,15


Next we compute the predictions at `newX`, and view them by extracting the `predicted_mean` attribute.

In [27]:
new_predictions = results.get_prediction(newX)
new_predictions.predicted_mean

array([29.80359411, 25.05334734, 20.30310057])

We can also product confidence  (i.e., intervals around the true value of $f(x_i)$ for each of our $x_i$ where $f$ is the true model for $y = f(X) + \epsilon$) for the predicted values.

In [30]:
new_predictions.conf_int(alpha=0.05)

array([[29.00741194, 30.59977628],
       [24.47413202, 25.63256267],
       [19.73158815, 20.87461299]])

To get prediction intervals (i.e., intervals around the true value of $y$ where $y = f(X) + \epsilon$) we set `obs=True`. Note that these are larger than the confidence intervals, accounting for the error term $\epsilon$.

In [32]:
new_predictions.conf_int(obs=True, alpha=0.05)

array([[17.56567478, 42.04151344],
       [12.82762635, 37.27906833],
       [ 8.0777421 , 32.52845905]])

The next task is to plot `medv` and `lstat` using `DataFrame.plot.scatter()`, and add the regression line to the resulting plot.